In [1]:
# Install specific versions of NumPy and TensorFlow
!pip install numpy==1.23.5  # Downgrade to a compatible version
!pip install tensorflow==2.12.0  # Use a compatible version of TensorFlow
!pip install transformers==4.25.1  # Ensure compatibility with TensorFlow
!pip install datasets torch  # Additional libraries

  Using cached tensorflow-2.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached tensorboard-2.12.3-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached wrapt-1.14.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
  Using cached jax-0.4.34-py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
cd MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

# Load your dataset
df = pd.read_csv('Conversation.csv')

# Prepare dataset for training
dataset = Dataset.from_pandas(df[['question', 'answer']])

In [6]:
from transformers import GPT2Tokenizer

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Corrected tokenize function
def tokenize_function(examples):
    # Combine question and answer for each example
    combined_texts = [q + " " + a + tokenizer.eos_token for q, a in zip(examples['question'], examples['answer'])]
    return tokenizer(combined_texts, truncation=True, padding='max_length', max_length=100)

# Tokenize data
tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/3727 [00:00<?, ? examples/s]

In [7]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Update the tokenize function to include labels for training
def tokenize_function(examples):
    combined_texts = [q + " " + a + tokenizer.eos_token for q, a in zip(examples['question'], examples['answer'])]
    # Tokenize the combined text
    tokenized_inputs = tokenizer(combined_texts, truncation=True, padding='max_length', max_length=100)
    # Set the labels to be the same as the input_ids for training (shifted language model task)
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'].copy()
    return tokenized_inputs

# Apply tokenization to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # Change "evaluation_strategy" to "eval_strategy" in the latest versions if required.
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # Use the same dataset for simplicity; ideally, separate train/test sets
)

# Train the model
trainer.train()

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Map:   0%|          | 0/3727 [00:00<?, ? examples/s]

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: question, answer. If question, answer are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3727
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 5592
  Number of trainable parameters = 124439808


Epoch,Training Loss,Validation Loss
1,0.467800,0.389430
2,0.399500,0.341707
3,0.362600,0.324405


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: question, answer. If question, answer are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3727
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in

TrainOutput(global_step=5592, training_loss=0.42604400023540884, metrics={'train_runtime': 844.7818, 'train_samples_per_second': 13.235, 'train_steps_per_second': 6.619, 'total_flos': 570606681600000.0, 'train_loss': 0.42604400023540884, 'epoch': 3.0})

In [8]:

# Define the output directory where the model will be saved
output_dir = "./fine_tuned_model"

# Save the fine-tuned model, tokenizer, and training arguments
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Configuration saved in ./fine_tuned_model/config.json
Model weights saved in ./fine_tuned_model/pytorch_model.bin
tokenizer config file saved in ./fine_tuned_model/tokenizer_config.json
Special tokens file saved in ./fine_tuned_model/special_tokens_map.json


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json')

In [13]:
import torch

def generate_response(prompt):
    # Move model to the appropriate device (CPU or GPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Encode the input prompt and move input_ids to the same device as the model
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Generate the response
    output = model.generate(
        input_ids,
        max_length=150,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        do_sample=True,
        no_repeat_ngram_size=3  # Prevents repeating sequences of 3 words
    )

    # Decode and return the generated response
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test the chatbot with a sample prompt
prompt = "ramkumar"
response = generate_response(prompt)
print("Bot:", response)

Bot: ramkumar's brother, who died, was one of the first soldiers in the army. what happened to him?


In [24]:
import torch

def generate_response(prompt):
    # Move model to the appropriate device (CPU or GPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Encode the input prompt and move input_ids to the same device as the model
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Generate the response
    output = model.generate(
        input_ids,
        max_length=150,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        top_k=20,  # Limit to the top 20 tokens (less random)
        top_p=0.9,  # Focus on the top 90% of the probability mass (more deterministic)
        temperature=0.5,  # Lower temperature for less randomness
        do_sample=False,  # Disable sampling for deterministic responses
        num_beams=5,  # Use beam search with 5 beams for better accuracy
        no_repeat_ngram_size=3  # Prevents repeating sequences of 3 words
    )

    # Decode and return the generated response
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test the chatbot with a sample prompt
prompt = "ramkumar"
response = generate_response(prompt)
print("Bot:", response)

Bot: ramkumar is a very nice man. he has a good nose.


In [30]:
%%writefile app.py
import streamlit as st
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('./fine_tuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_model')

# Function to generate a response from the model
def generate_response(prompt):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    output = model.generate(
        input_ids,
        max_length=150,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        top_k=10,  # Lower top_k for more focused results
        top_p=0.7,  # Lower top_p for more deterministic results
        temperature=0.5,  # Lower temperature for more precise responses
        do_sample=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Streamlit app layout
st.title("AI Chatbot")
user_input = st.text_input("You:")

if user_input:
    response = generate_response(user_input)
    st.write("Bot:", response)

Writing app.py


In [23]:
!npm install -g localtunnel


added 22 packages, and audited 23 packages in 4s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [31]:
!pip install streamlit

In [32]:
!streamlit run app.py --server.port 8501 & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.171.229.166:8501

Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y)   Stopping...
^C


SyntaxError: invalid syntax (<ipython-input-36-cd3235798d58>, line 1)